In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, InputLayer, Dropout, Dense, Flatten, Embedding
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
import tensorflow as tf

from tensorflow.keras.layers import concatenate
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('data/affcon_final_linguistics.csv')

In [5]:
# Categorical codes for deception quadrant

train, test = train_test_split(df, test_size=0.2)

y_train_deception = to_categorical(train['Input.deception_quadrant_cat'].tolist())
y_train_rapport = to_categorical(train['affcon_rapport'].tolist())
y_train_share_information = to_categorical(train['affcon_shareinformation'].tolist())
y_train_reasoning = to_categorical(train['affcon_reasoning'].tolist())
y_train_gamemove = to_categorical(train['affcon_gamemove'].tolist())

y_test_deception = to_categorical(test['Input.deception_quadrant_cat'].tolist())
y_test_rapport = to_categorical(test['affcon_rapport'].tolist())
y_test_share_information = to_categorical(test['affcon_shareinformation'].tolist())
y_test_reasoning = to_categorical(test['affcon_reasoning'].tolist())
y_test_gamemove = to_categorical(test['affcon_gamemove'].tolist())

X_train = train.drop(columns=['affcon_gamemove', 'affcon_reasoning',
                              'affcon_rapport', 'affcon_shareinformation', 'Input.deception_quadrant',
                              'Input.deception_quadrant_cat'])

X_test = test.drop(columns=['affcon_gamemove', 'affcon_reasoning',
                              'affcon_rapport', 'affcon_shareinformation', 'Input.deception_quadrant',
                              'Input.deception_quadrant_cat'])

# X_train = train.drop(columns=['affcon_rapport', 'Input.full_text',
#                                     'msg_id', 'Input.convo_id', 'Input.train_test_val',
#                                      'Input.msg_id', 'Input.timestamp', 'Input.full_text',
#                                      'affcon_gamemove', 'affcon_reasoning', 'affcon_rapport',
#                                      'affcon_shareinformation', 'Input.speaker', 'Input.reply_to',
#                                      'Input.speaker_intention', 'Input.reciever_perception',
#                                      'Input.reciever', 'Input.absolute_message_index', 
#                                      'Input.relative_message_index', 'Input.year', 'Input.game_score_speaker',
#                                      'Input.game_score_receiver', 'Input.game_score_delta',
#                                      'Input.deception_quadrant', 'Input.num_words', 
#                                      'Input.num_characters', 'Input.sno', 'Input.sno1', 'Input.deception_quadrant_codes'
#                                     ])

# X_test = test.drop(columns=['affcon_rapport', 'Input.full_text',
#                                     'msg_id', 'Input.convo_id', 'Input.train_test_val',
#                                      'Input.msg_id', 'Input.timestamp', 'Input.full_text',
#                                      'affcon_gamemove', 'affcon_reasoning', 'affcon_rapport',
#                                      'affcon_shareinformation', 'Input.speaker', 'Input.reply_to',
#                                      'Input.speaker_intention', 'Input.reciever_perception',
#                                      'Input.reciever', 'Input.absolute_message_index', 
#                                      'Input.relative_message_index', 'Input.year', 'Input.game_score_speaker',
#                                      'Input.game_score_receiver', 'Input.game_score_delta',
#                                      'Input.deception_quadrant', 'Input.num_words', 
#                                      'Input.num_characters', 'Input.sno', 'Input.sno1', 'Input.deception_quadrant_codes'
#                                     ])

In [6]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)

Using TensorFlow backend.


In [22]:
# Game move model
inputB = Input(shape=(X_train.shape[1],))
c = Dense(2, activation='relu')(inputB)
c = Dense(4, activation='relu')(c)
c = Dense(2, activation='linear')(c)
gamemove_model = Model(inputs=inputB, outputs=c)

gamemove_model.compile(loss='binary_crossentropy', optimizer='adam', 
                      metrics=['acc',f1_m,precision_m, recall_m])
history = gamemove_model.fit(x=X_train, y=y_train_gamemove, epochs=32, 
                    batch_size=64, 
                    validation_split=0.2, callbacks=[callback])
loss, accuracy, f1_score, precision, recall = gamemove_model.evaluate(X_test, y_test_gamemove, verbose=0)
print(precision, recall, f1_score)

Epoch 1/32
40/40 [==============================] - 0s 10ms/step - loss: 7.7125 - acc: 0.7105 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 7.7125 - val_acc: 0.7460 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 2/32
40/40 [==============================] - 0s 4ms/step - loss: 7.7125 - acc: 0.7220 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 7.7125 - val_acc: 0.7460 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
0.0 0.0 0.0


In [23]:
# Reasoning model
inputB = Input(shape=(X_train.shape[1],))
c = Dense(2, activation='relu')(inputB)
c = Dense(4, activation='relu')(c)
c = Dense(2, activation='linear')(c)
reasoning_model = Model(inputs=inputB, outputs=c)

reasoning_model.compile(loss='binary_crossentropy', optimizer='adam', 
                      metrics=['acc',f1_m,precision_m, recall_m])
history = reasoning_model.fit(x=X_train, y=y_train_reasoning, epochs=32, 
                    batch_size=64, 
                    validation_split=0.2, callbacks=[callback])
loss, accuracy, f1_score, precision, recall = reasoning_model.evaluate(X_test, y_test_reasoning, verbose=0)
print(precision, recall, f1_score)

Epoch 1/32
40/40 [==============================] - 0s 9ms/step - loss: 7.7627 - acc: 0.4539 - f1_m: 0.0292 - precision_m: 0.2606 - recall_m: 0.0156 - val_loss: 7.7910 - val_acc: 0.4889 - val_f1_m: 0.0064 - val_precision_m: 0.0700 - val_recall_m: 0.0034
Epoch 2/32
40/40 [==============================] - 0s 4ms/step - loss: 7.7232 - acc: 0.4925 - f1_m: 0.0187 - precision_m: 0.2979 - recall_m: 0.0098 - val_loss: 7.7758 - val_acc: 0.5143 - val_f1_m: 0.0036 - val_precision_m: 0.1000 - val_recall_m: 0.0019
Epoch 3/32
40/40 [==============================] - 0s 4ms/step - loss: 7.7054 - acc: 0.5481 - f1_m: 0.0143 - precision_m: 0.2667 - recall_m: 0.0074 - val_loss: 7.7750 - val_acc: 0.5460 - val_f1_m: 0.0036 - val_precision_m: 0.1000 - val_recall_m: 0.0019
Epoch 4/32
40/40 [==============================] - 0s 4ms/step - loss: 7.7020 - acc: 0.5659 - f1_m: 0.0129 - precision_m: 0.2808 - recall_m: 0.0066 - val_loss: 7.7631 - val_acc: 0.5460 - val_f1_m: 0.0036 - val_precision_m: 0.1000 - val_r

In [24]:
# Share Information model
inputB = Input(shape=(X_train.shape[1],))
c = Dense(2, activation='relu')(inputB)
c = Dense(4, activation='relu')(c)
c = Dense(2, activation='linear')(c)
shareinfo_model = Model(inputs=inputB, outputs=c)

shareinfo_model.compile(loss='binary_crossentropy', optimizer='adam', 
                      metrics=['acc',f1_m,precision_m, recall_m])
history = shareinfo_model.fit(x=X_train, y=y_train_share_information, epochs=32, 
                    batch_size=64, 
                    validation_split=0.2, callbacks=[callback])
loss, accuracy, f1_score, precision, recall = shareinfo_model.evaluate(X_test, y_test_share_information, verbose=0)
print(precision, recall, f1_score)

Epoch 1/32
40/40 [==============================] - 0s 10ms/step - loss: 7.8514 - acc: 0.4932 - f1_m: 0.2336 - precision_m: 0.4394 - recall_m: 0.1617 - val_loss: 7.7747 - val_acc: 0.4921 - val_f1_m: 0.2121 - val_precision_m: 0.4563 - val_recall_m: 0.1388
Epoch 2/32
40/40 [==============================] - 0s 4ms/step - loss: 7.7271 - acc: 0.4913 - f1_m: 0.1689 - precision_m: 0.4506 - recall_m: 0.1051 - val_loss: 7.7356 - val_acc: 0.4968 - val_f1_m: 0.1930 - val_precision_m: 0.4538 - val_recall_m: 0.1229
Epoch 3/32
40/40 [==============================] - 0s 4ms/step - loss: 7.6353 - acc: 0.5064 - f1_m: 0.2402 - precision_m: 0.4619 - recall_m: 0.1667 - val_loss: 6.5315 - val_acc: 0.4889 - val_f1_m: 0.3329 - val_precision_m: 0.4974 - val_recall_m: 0.2512
Epoch 4/32
40/40 [==============================] - 0s 4ms/step - loss: 6.3627 - acc: 0.4754 - f1_m: 0.2505 - precision_m: 0.4595 - recall_m: 0.1750 - val_loss: 6.1609 - val_acc: 0.4889 - val_f1_m: 0.2730 - val_precision_m: 0.4900 - val_

In [27]:
# Rapport model
inputB = Input(shape=(X_train.shape[1],))
c = Dense(2, activation='relu')(inputB)
c = Dense(4, activation='relu')(c)
c = Dense(2, activation='linear')(c)
rapport_model = Model(inputs=inputB, outputs=c)

rapport_model.compile(loss='binary_crossentropy', optimizer='adam', 
                      metrics=['acc',f1_m,precision_m, recall_m])
history = rapport_model.fit(x=X_train, y=y_train_rapport, epochs=32, 
                    batch_size=64, 
                    validation_split=0.2, callbacks=[callback])
loss, accuracy, f1_score, precision, recall = rapport_model.evaluate(X_test, y_test_rapport, verbose=0)
print(precision, recall, f1_score)

Epoch 1/32
40/40 [==============================] - 0s 8ms/step - loss: 6.6143 - acc: 0.4095 - f1_m: 0.6251 - precision_m: 0.4980 - recall_m: 0.8408 - val_loss: 6.4190 - val_acc: 0.3905 - val_f1_m: 0.6106 - val_precision_m: 0.4939 - val_recall_m: 0.8028
Epoch 2/32
40/40 [==============================] - 0s 3ms/step - loss: 6.2351 - acc: 0.3963 - f1_m: 0.6096 - precision_m: 0.4965 - recall_m: 0.7912 - val_loss: 6.2733 - val_acc: 0.3905 - val_f1_m: 0.6058 - val_precision_m: 0.4943 - val_recall_m: 0.7856
Epoch 3/32
40/40 [==============================] - 0s 3ms/step - loss: 6.1657 - acc: 0.3963 - f1_m: 0.6067 - precision_m: 0.4959 - recall_m: 0.7833 - val_loss: 6.1478 - val_acc: 0.3905 - val_f1_m: 0.5962 - val_precision_m: 0.4904 - val_recall_m: 0.7637
Epoch 4/32
40/40 [==============================] - 0s 4ms/step - loss: 5.9300 - acc: 0.3963 - f1_m: 0.5966 - precision_m: 0.4956 - recall_m: 0.7509 - val_loss: 5.9299 - val_acc: 0.3905 - val_f1_m: 0.5797 - val_precision_m: 0.4863 - val_r

In [30]:
from keras.layers import * 

commonInput = tf.keras.layers.Input(shape=(X_train.shape[1],))

out1 = gamemove_model(commonInput)    
out2 = reasoning_model(commonInput)   
out3 = shareinfo_model(commonInput)
mergedTwo = tf.keras.layers.Add()([out1,out2])
mergedOut = tf.keras.layers.Subtract()([mergedTwo, out3])

mergedOut = tf.keras.layers.Flatten()(mergedOut)    
mergedOut = tf.keras.layers.Dense(256, activation='relu')(mergedOut)
mergedOut = tf.keras.layers.Dropout(.5)(mergedOut)
mergedOut = tf.keras.layers.Dense(128, activation='relu')(mergedOut)
mergedOut = tf.keras.layers.Dropout(.35)(mergedOut)
mergedOut = tf.keras.layers.Dense(2, activation='softmax')(mergedOut)  #Cuz binary

mergedModel = tf.keras.Model(commonInput, mergedOut)
mergedModel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc',f1_m,precision_m, recall_m])
mergedModel.fit(x=X_train, y=y_train_rapport, epochs=8, batch_size=64, 
                    validation_split=0.2, callbacks=[callback])

Epoch 1/8
40/40 [==============================] - 0s 11ms/step - loss: 0.7579 - acc: 0.5655 - f1_m: 0.5667 - precision_m: 0.5667 - recall_m: 0.5667 - val_loss: 0.6723 - val_acc: 0.6095 - val_f1_m: 0.6107 - val_precision_m: 0.6107 - val_recall_m: 0.6107
Epoch 2/8
40/40 [==============================] - 0s 5ms/step - loss: 0.6877 - acc: 0.5953 - f1_m: 0.5945 - precision_m: 0.5945 - recall_m: 0.5945 - val_loss: 0.6720 - val_acc: 0.6095 - val_f1_m: 0.6107 - val_precision_m: 0.6107 - val_recall_m: 0.6107
Epoch 3/8
40/40 [==============================] - 0s 5ms/step - loss: 0.6769 - acc: 0.6025 - f1_m: 0.6053 - precision_m: 0.6053 - recall_m: 0.6053 - val_loss: 0.6708 - val_acc: 0.6095 - val_f1_m: 0.6107 - val_precision_m: 0.6107 - val_recall_m: 0.6107
Epoch 4/8
40/40 [==============================] - 0s 5ms/step - loss: 0.6778 - acc: 0.6021 - f1_m: 0.6041 - precision_m: 0.6041 - recall_m: 0.6041 - val_loss: 0.6715 - val_acc: 0.6095 - val_f1_m: 0.6107 - val_precision_m: 0.6107 - val_reca

### Stacked Random Forest 

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [11]:
# Game Move Random Forest
clf_gamemove = RandomForestClassifier(max_depth=2, random_state=0)
clf_gamemove.fit(X_train, y_train_gamemove)
y_pred = clf_gamemove.predict(X_test)
print(classification_report(y_test_gamemove, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       880
           1       0.72      1.00      0.84      2268

   micro avg       0.72      0.72      0.72      3148
   macro avg       0.36      0.50      0.42      3148
weighted avg       0.52      0.72      0.60      3148
 samples avg       0.72      0.72      0.72      3148



C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
# Share info Random Forest
clf_shareinfo = RandomForestClassifier(max_depth=2, random_state=0)
clf_shareinfo.fit(X_train, y_train_share_information)
y_pred = clf_shareinfo.predict(X_test)
print(classification_report(y_test_share_information, y_pred))

              precision    recall  f1-score   support

           0       0.66      0.23      0.34      1534
           1       0.55      0.89      0.68      1614

   micro avg       0.56      0.56      0.56      3148
   macro avg       0.60      0.56      0.51      3148
weighted avg       0.60      0.56      0.51      3148
 samples avg       0.56      0.56      0.56      3148



In [11]:
# Reasoning Random Forest
clf_reasoning = RandomForestClassifier(max_depth=2, random_state=0)
clf_reasoning.fit(X_train, y_train_reasoning)
y_pred = clf_reasoning.predict(X_test)
print(classification_report(y_test_reasoning, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       727
           1       0.77      1.00      0.87      2421

   micro avg       0.77      0.77      0.77      3148
   macro avg       0.38      0.50      0.43      3148
weighted avg       0.59      0.77      0.67      3148
 samples avg       0.77      0.77      0.77      3148



C:\Users\lynne\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Rapport Random Forest
clf_rapport = RandomForestClassifier(max_depth=2, random_state=0)
clf_rapport.fit(X_train, y_train_rapport)
y_pred = clf_rapport.predict(X_test)
print(classification_report(y_test_rapport, y_pred))

In [17]:
# Output Dataframe 
one_hot = []
for i in range(0, len(y_train_gamemove)):
    one_hot_obj = {}
    one_hot_obj['gamemove'] = np.argmax(y_train_gamemove[i])
    one_hot_obj['reasoning'] = np.argmax(y_train_reasoning[i])
    one_hot_obj['rapport'] = np.argmax(y_train_rapport[i])
    one_hot_obj['shareinfo'] = np.argmax(y_train_share_information[i])
    one_hot.append(one_hot_obj)
    
one_hot_df = pd.DataFrame(one_hot)

In [43]:
y_pred_reasoning = clf_reasoning.predict(X_train)
y_pred_gamemove = clf_reasoning.predict(X_train)

In [18]:
inputB = Input(shape=(one_hot_df.shape[1],))
c = Dense(2, activation='relu')(inputB)
c = Dense(4, activation='relu')(c)
c = Dense(2, activation='linear')(c)
m = Model(inputs=inputB, outputs=c)

m.compile(loss='binary_crossentropy', optimizer='adam', 
                      metrics=['acc',f1_m,precision_m, recall_m])
history = m.fit(x=one_hot_df, y=y_train_deception, epochs=32, 
                    batch_size=64, 
                    validation_split=0.2, callbacks=[callback])
#loss, accuracy, f1_score, precision, recall = m.evaluate(X_test, y_test_rapport, verbose=0)
#print(precision, recall, f1_score)

Epoch 1/32
158/158 [==============================] - 1s 4ms/step - loss: 7.7125 - acc: 0.8986 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 7.7125 - val_acc: 0.9003 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 2/32
158/158 [==============================] - 1s 3ms/step - loss: 7.7125 - acc: 0.8986 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 7.7125 - val_acc: 0.9003 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
